In [21]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np

url = 'https://racetozerodataexplorer.org/proceed/'

names = []
e1 = []
e2l = []
e2m = []
e_master = [e1, e2l, e2m]

non_click = ['-', '-', '-', '-']

with webdriver.Chrome() as driver:
    driver.get(url)
    table = driver.find_elements(By.XPATH, "//*[@class= 'styles_table__jWPKo']/tbody/tr")
    for row in table:
        name = row.find_element(By.TAG_NAME, 'th')
        names.append(name.text)

        for i, cell in enumerate(row.find_elements(By.TAG_NAME, 'td')):
            if i % 9 in [0,1,2]:
                try:
                    cell.find_element(By.CLASS_NAME, 'styles_button__IS06I').click()
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'styles_axisValue__ipOq1')))
                    values = driver.find_elements(By.CLASS_NAME, 'styles_axisValue__ipOq1')
                    e_master[i].append([value.text.replace(',','') for value in values])
                    driver.find_element(By.CLASS_NAME, 'styles_close__cpVjT').click()
                except:
                    e_master[i].append(non_click)
                    continue

In [24]:
df1 = pd.DataFrame(e1, columns=['E1-2018', 'E1-2019', 'E1-2020', 'E1-2021'])
df2 = pd.DataFrame(e2l, columns=['E2L-2018', 'E2L-2019', 'E2L-2020', 'E2L-2021'])
df3 = pd.DataFrame(e2m, columns=['E2M-2018', 'E2M-2019', 'E2M-2020', 'E2M-2021'])

df = pd.concat([df1, df2, df3], axis=1)
df.insert(0, 'Company', names)
df.to_csv('emissions.csv', index=False)
df

,Company,E1-2018,E1-2019,E1-2020,E1-2021,E2L-2018,E2L-2019,E2L-2020,E2L-2021,E2M-2018,E2M-2019,E2M-2020,E2M-2021
0,37 Interactive Entertainment,-,129,169,152,-,2701,2395,1794,-,-,-,-
1,A.P. Moller - Maersk,38749000,36204000,33902000,36863000,338000,287000,289000,310170,-,-,337000,330182
2,AAK AB,-,-,-,269380,-,-,-,18632,-,-,-,74351
3,AB Volvo,-,245000,205000,245000,-,215000,193000,204000,-,124000,121000,114000
4,ABB,375000,257500,225547,190900,650799,456000,414000,370000,627903,410800,335300,214000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,"Yum China Holdings, Inc.",-,114720,135615,158887,-,1956072,1873757,2158856,-,-,-,-
496,"Yum! Brands, Inc.",48047,23545,38907,39013,155439,140341,128129,117621,159403,144313,130953,112665
497,"Yutong Bus Co., Ltd.",-,-,-,-,-,-,-,-,-,-,-,-
498,Żabka Polska Sp. z o. o.,-,-,-,7653,-,-,-,-,-,-,-,-
